# Examen práctico 
## Parcial 2 Laboratorio de Aprendizaje Estadístico
Diego Lozoya Morales | 745345

# Paqueterías

# Funciones

# Data

## Variable dependiente

# Análisis Dataset

## Correlación

## Análisis vibariado

# Feature engineering

## Dummies

## Transformaciones

## Nuevas variables

# Significancia de variables

# Preparación de datos

## Cross validation

# KNN clasificador

# Regresión logística

# Red neuronal

# Discriminante lineal

# Conlusiones generales